In [ ]:
import json, pathlib, itertools
p=pathlib.Path('artifacts/level3_llm_run/llm_parsed')
if not p.exists():
    print("MISSING:", p); raise SystemExit(1)
lines=[json.loads(l) for l in p.read_text().splitlines() if l.strip()]
total=len(lines)
raws=[l.get('raw_output','') for l in lines]
cnt_ollama_err=sum(1 for r in raws if isinstance(r,str) and r.startswith('__OLLAMA_ERROR__'))
parseable=0; objs=0; arrays=0; nonparse=[]
for i,r in enumerate(raws):
    if not isinstance(r,str):
        nonparse.append((i,str(type(r))))
        continue
    try:
        parsed=json.loads(r)
        parseable+=1
        if isinstance(parsed, list):
            arrays+=1
        elif isinstance(parsed, dict):
            objs+=1
    except Exception:
        nonparse.append((i, (r[:400].replace('\\n',' ').replace('\\r',' '))))
print("total_calls:", total)
print("__OLLAMA_ERROR__ count:", cnt_ollama_err)
print("parseable JSON:", parseable, "objects:", objs, "arrays:", arrays, "non-JSON/failed:", len(nonparse))
print("\\nExamples of non-JSON/raw outputs (up to 5):")
for i,s in itertools.islice(nonparse,5):
    print(f"index={i}: {s}\\n")
print("\\nFirst 5 raw outputs (full lines):")
for l in lines[:5]:
    print(json.dumps(l, indent=2)[:1000])

total_calls: 201
__OLLAMA_ERROR__ count: 200
parseable JSON: 1 objects: 1 arrays: 0 non-JSON/failed: 200
\nExamples of non-JSON/raw outputs (up to 5):
index=0: __OLLAMA_ERROR__ RuntimeError('ollama run/generate/chat attempts failed. Details: cmd=[\'ollama\', \'run\', \'llama2\'] rc=timeout err=command timed out cmd=[\'ollama\', \'generate\', \'llama2\', \'--temperature\', \'0\'] rc=1 err=Error: unknown command "generate" for "ollama" cmd=[\'ollama\', \'chat\', \'llama2\', \'--no-stream\'] rc=1 err=Error: unknown command "chat" for "ollama"')\n
index=1: __OLLAMA_ERROR__ RuntimeError('ollama run/generate/chat attempts failed. Details: cmd=[\'ollama\', \'run\', \'llama2\'] rc=timeout err=command timed out cmd=[\'ollama\', \'generate\', \'llama2\', \'--temperature\', \'0\'] rc=1 err=Error: unknown command "generate" for "ollama" cmd=[\'ollama\', \'chat\', \'llama2\', \'--no-stream\'] rc=1 err=Error: unknown command "chat" for "ollama"')\n
index=2: __OLLAMA_ERROR__ RuntimeError('ollama run/

In [2]:
import pandas as pd 
p='artifacts/level3_llm_run/llm_parsed.parquet' 
df=pd.read_parquet(p)
print("rows:", len(df))
print("llm_adjustment_valid:", int(df['llm_adjustment_valid'].sum()))
print("needs_review:", int(df['needs_review'].sum()))
print("non-zero clamped:", int((df['llm_adjustment_clamped']!=0).sum()))
print(df['llm_adjustment_clamped'].describe().to_string())

rows: 100
llm_adjustment_valid: 0
needs_review: 100
non-zero clamped: 0
count    100.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0


In [3]:
import json, pathlib
p=pathlib.Path('artifacts/level3_llm_run/llm_raw.jsonl')
for i,l in enumerate(p.read_text().splitlines()):
    j=json.loads(l)
    ro=j.get('raw_output','')
    if isinstance(ro,str) and not ro.startswith('__OLLAMA_ERROR__'):
        print("INDEX",i)
        print(json.dumps(j, indent=2, ensure_ascii=False))
        break
print("done")

INDEX 100
{
  "transaction_id": "WARMUP",
  "timestamp_utc": "2026-01-25T16:15:30.880384+00:00",
  "prompt": "<warmup prompt>",
  "raw_output": "{\n  \"transaction_id\": \"WARMUP\",\n  \"llm_adjustment\": 0.0,\n  \"evidence_ids\": [\"E1\"],\n  \"explanation\": \"baseline=0.0000\",\n  \"confidence\": null\n}\n\n",
  "model": "llama2",
  "elapsed_seconds": 30.193673133850098
}
done


In [4]:
import pandas as pd 
res = pd.read_parquet('artifacts/results_stream.parquet') 
llm = pd.read_parquet('artifacts/level3_llm_run/llm_parsed.parquet') 
res['transaction_id']=res['transaction_id'].astype(str) 
llm['transaction_id']=llm['transaction_id'].astype(str) 
llm_sel = llm[['transaction_id','llm_adjustment_clamped','llm_adjustment_raw','llm_adjustment_valid','llm_evidence_ids','needs_review']].copy() 
merged = res.merge(llm_sel, on='transaction_id', how='left') 
merged['llm_adjustment_clamped'] = merged['llm_adjustment_clamped'].fillna(0.0) 
merged['llm_adjustment_valid'] = merged['llm_adjustment_valid'].fillna(False) 
merged.to_parquet('artifacts/results_stream_with_llm.parquet', index=False) 
print("WROTE artifacts/results_stream_with_llm.parquet")

WROTE artifacts/results_stream_with_llm.parquet
